In [0]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import time
import math
import random

In [0]:
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!ls "/content/drive/My Drive/ISP2020/Data/Skels"

Clap_skels  Robot_skels  Snoop_skels  Up_clap_skels


In [0]:
LABELS = [
         # "Snoop", #0
          "Robot", #1
          "Clap" , #2
         # "Up_clap" #3
          ]

n_steps = 32
n_categories = len(LABELS)

Data loading


In [0]:
def read_from_file(path):
    with open(path, 'r') as file:
        data = file.read()
        cleared_data = ''
        for let in data:
            if let.isdigit() or let == '-':
                cleared_data+=let
            else:
                cleared_data+=','
        cleared_data = cleared_data.split(',')
        data = [int(i) for i in cleared_data if i] 
        data = np.asarray(data)
        data = data.reshape(-1,36)
    return data
    

In [0]:
def get_data(path, label):
  data = read_from_file(path)
  blocks = int(len(data) / n_steps)
  train_data_ = data[:int(blocks * 0.8)  * n_steps]
  test_data_ = data[int(blocks * 0.8)  * n_steps : blocks * n_steps]
  
  train_data = np.array(np.split(train_data_,int(train_data_.shape[0]/n_steps)))
  test_data = np.array(np.split(test_data_,int(test_data_.shape[0]/n_steps)))
  
  y_train = np.array(np.split(label*np.ones(train_data.shape[0]),  train_data.shape[0]))
  y_test = np.array(np.split(label*np.ones(test_data.shape[0]),  test_data.shape[0])) 

  return train_data, test_data, y_train, y_test

  


In [0]:
train_data, test_data, y_train, y_test = get_data("/content/drive/My Drive/ISP2020/Data/Skels/Robot_skels/skel_robot_osokin.txt", 0)

In [0]:
X_train, X_test, y_train, y_test = get_data("/content/drive/My Drive/ISP2020/Data/Skels/Clap_skels/skel_clap_osokin.txt", 1)

In [0]:
X_train = np.concatenate((X_train, train_data), axis = 0)
X_test = np.concatenate((X_test, test_data), axis = 0)

y_train = np.concatenate((y_train, y_train_), axis = 0)
y_test = np.concatenate((y_test, y_test_), axis = 0)

In [0]:
y_train_osokin = np.array(np.split(np.ones(train_data_clap_osokin.shape[0]),  train_data_clap_osokin.shape[0]))
y_test_osokin = np.array(np.split(np.ones(test_data_clap_osokin.shape[0]),  test_data_clap_osokin.shape[0]))

In [0]:
X_train = np.array(np.concatenate((X_train_, train_data_clap_osokin),axis = 0 ))
X_test = np.array(np.concatenate((X_test_, test_data_clap_osokin),axis = 0 ))

y_train = np.array(np.concatenate((y_train_, y_train_osokin),axis = 0 ))
y_test = np.array(np.concatenate((y_train_, y_test_osokin),axis = 0 ))

In [0]:
X_train.shape

(36, 32, 36)

In [0]:
y_train

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

Task. Create the function, what can create train and test data from all skeletons in all folders. 

Some hints: 
* np.concatenate() will help you to join data and labels (axis = None)
* np.ones(n) * 2 - array of number 2 (it's for labeling)


In [0]:
tensor_X_test = torch.from_numpy(X_test).float()
print('test_data_size:',tensor_X_test.size())
tensor_y_test = torch.from_numpy(y_test).float()
# tensor_y_test = torch.unsqueeze(tensor_y_test, 1)
print('test_label_size:',tensor_y_test.size())
n_data_size_test = tensor_X_test.size()[0]
print('n_data_size_test:',n_data_size_test)

tensor_X_train = torch.from_numpy(X_train).float()
print('train_data_size:',tensor_X_train.size())
tensor_y_train = torch.from_numpy(y_train).float()
# tensor_y_train = torch.unsqueeze(tensor_y_train, 1)
print('train_label_size:',tensor_y_train.size())
n_data_size_train = tensor_X_train.size()[0]
print('n_data_size_train:',n_data_size_train)

NameError: ignored

In [0]:
class LSTM(nn.Module):
    
    def __init__(self,input_dim,hidden_dim,output_dim,layer_num):
        super(LSTM,self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.lstm = torch.nn.LSTM(input_dim,hidden_dim,layer_num,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim,output_dim)
        self.bn = nn.BatchNorm1d(32)
        
    def forward(self,inputs):
        x = self.bn(inputs)
        lstm_out,(hn,cn) = self.lstm(x)
        out = self.fc(lstm_out[:,-1,:])
        return out

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [0]:
n_hidden = 128
n_joints = 18*2
n_categories = 2
n_layer = 3
rnn = LSTM(n_joints,n_hidden,n_categories,n_layer)
rnn.to(device)

RuntimeError: ignored

In [0]:
import random
def randomTrainingExampleBatch(batch_size,flag,num=-1):
    if flag == 'train':
        X = tensor_X_train
        y = tensor_y_train
        data_size = n_data_size_train
    elif flag == 'test':
        X = tensor_X_test
        y = tensor_y_test
        data_size = n_data_size_test
    if num == -1:
        ran_num = random.randint(0,data_size-batch_size)
    else:
        ran_num = num
    pose_sequence_tensor = X[ran_num:(ran_num+batch_size)]
    pose_sequence_tensor = pose_sequence_tensor
    category_tensor = y[ran_num:ran_num+batch_size,:]
    return category_tensor.long(),pose_sequence_tensor


In [0]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return LABELS[category_i], category_i

In [0]:
import torch.optim as optim
import time
import math

criterion = nn.CrossEntropyLoss()
learning_rate = 0.0005
optimizer = optim.SGD(rnn.parameters(),lr=learning_rate,momentum=0.9)
#scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10000, gamma=0.1)

n_iters = 10000
print_every = 100
plot_every = 100
batch_size = 4

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
   
    category_tensor, input_sequence = randomTrainingExampleBatch(batch_size,'train')
    input_sequence = input_sequence.to(device)
    category_tensor = category_tensor.to(device)
    category_tensor = torch.squeeze(category_tensor)
    
    optimizer.zero_grad()
    
    output = rnn(input_sequence)
    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step() 
    #scheduler.step()
    
    #如果tensor只有一个元素,那么可以用.item()的方法取出这个元素作为普通的python数字.
    current_loss += loss.item()
    
    category = LABELS[int(category_tensor[0])]

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f  / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, guess, correct))
        
    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

RuntimeError: ignored